In [43]:
from google.cloud import bigquery
from google.cloud import language
import pandas as pd
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'C:\\Users\\antho\\Downloads\\ADAproj-02b470a29ee4.json'#
# get json by following (Setting up authentication)
# https://cloud.google.com/bigquery/docs/reference/libraries

In [2]:
bigquery_client = bigquery.Client()

# Getting the main tables:
- Protests for leadership change
- Protests for rights
- country code table
- event code table

In [47]:
event_codes = pd.read_csv("eventcodes.csv", encoding = "ISO-8859-1")

In [48]:
event_codes.head()

,CAMEOEVENTCODE,EVENTDESCRIPTION
0,1,MAKE PUBLIC STATEMENT
1,10,"Make statement, not specified below"
2,11,Decline comment
3,12,Make pessimistic comment
4,13,Make optimistic comment


In [39]:
query_leadership_change_protests = bigquery_client.query(
    """SELECT COUNT(EventCode) as nb_protests FROM `gdelt-bq.gdeltv2.events` 
    WHERE EventCode LIKE '1411' OR EventCode LIKE '1421' 
    OR EventCode LIKE '1431' OR EventCode LIKE '1441'""")
leadership_change_protests_df = query_leadership_change_protests.result().to_dataframe()

In [40]:
leadership_change_protests_df.head()

,nb_protests
0,14750


In [41]:
query_rights_protests = bigquery_client.query(
    """SELECT COUNT(EventCode) as nb_protests FROM `gdelt-bq.gdeltv2.events` 
    WHERE EventCode LIKE '1413' OR EventCode LIKE '1423' 
    OR EventCode LIKE '1433' OR EventCode LIKE '1443'""")
rights_protests_df = query_rights_protests.result().to_dataframe()

In [42]:
rights_protests_df.head()

,nb_protests
0,630


In [13]:
query_job = bigquery_client.query(
    """SELECT  as nb_protests FROM `gdelt-bq.extra.countryinfo`""")
results = query_job.result().to_dataframe()

In [14]:
results.head()

,nb_protests
0,245
